In [2]:
import torch.nn as nn
conv = nn.Conv2d(3, 16, kernel_size =3)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [3]:
conv.weight.shape

torch.Size([16, 3, 3, 3])

In [4]:
# padding 
conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)


In [5]:
#pooling

pool = nn.MaxPool2d(2)
# output = pool(img)

In [1]:
# subclassing으로 모델 계층 만들기
import torch.nn as nn

class mynet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act = nn.Tanh()
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32,2)

    def forward(self, x):
        out = self.pool(self.act(self.conv1(x)))
        out = self.pool(self.act(self.conv2(out)))
        out = out.view(-1, 8*8*8)
        out = self.act(self.fc1(out))
        out = self.fc2(out)
        return out
    

model = mynet()
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [2]:
# 신경망 훈련시켜보기f
from torchvision import transforms, datasets
import torch
import torch.optim as optim

transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

label_map = {0:0, 2:1}
class_name = ['airplane', 'bird']

cifar10 = datasets.CIFAR10('.', train=True, download=False, transform=transform)
cifar10_val = datasets.CIFAR10('.', train=False, download=False, transform=transform)

cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0,2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0,2]]


def traning_loop(epochs, optimizer, input_model, loss_func, input):
    for epoch in range(1, epochs+1):
        loss_train = 0.0
        for imgs, labels in input:
            output = input_model(imgs)
            loss = loss_func(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_train += loss.item()
    
    if epoch == 1 or epoch % 10 ==0:
        print('Epoch {}, Training loss {}', epoch, loss_train/len(input))




In [ ]:

model = mynet()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()
traning_loop(100, optimizer, model, loss_fn, train_loader)

In [ ]:
path = '/Users/'
# 모델 저장
torch.save(model.state_dict(), path + 'first_convnet_for_torch.pt')

# 모델 로드
mymodel = mynet()
mymodel.load_state_dict(torch.load(path + 'first_convnet_for_torch.pt'))

In [3]:
print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}") 

# for cuda
device1 = (torch.device('mps') if torch.cuda.is_available() else torch.device('cpu'))
print(device1)

# for m1 mps
device = (torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu'))
print(device)

MPS 장치를 지원하도록 build가 되었는가? True
MPS 장치가 사용 가능한가? True
cpu
mps


In [4]:
def traning_loop(epochs, optimizer, input_model, loss_func, input):
    for epoch in range(1, epochs+1):
        loss_train = 0.0
        for imgs, labels in input:
            imgs = imgs.to(device = device)
            labels = labels.to(device = device)
            output = input_model(imgs)
            loss = loss_func(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        loss_train += loss.item()
    
        if epoch == 1 or epoch % 10 ==0:
            print(f'Epoch {epoch}, Training loss {loss_train/len(input)}')

train_loader = torch.utils.data.DataLoader(cifar2, batch_size = 64, shuffle = True)
model = mynet().to(device=device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()
traning_loop(100, optimizer, model, loss_fn, train_loader)

Epoch 1, Training loss 0.0016377558753748608
Epoch 10, Training loss 0.0018247466558104109
Epoch 20, Training loss 0.0013926970730921266
Epoch 30, Training loss 0.00034714791520385987
Epoch 40, Training loss 5.127190594460554e-05
Epoch 50, Training loss 1.1166556186641857e-05
Epoch 60, Training loss 1.450032173733043e-05
Epoch 70, Training loss 3.856712815820412e-06
Epoch 80, Training loss 0.0002573950655141454
Epoch 90, Training loss 4.7895888305583575e-06
Epoch 100, Training loss 3.2131144907444145e-05
